
# 🧠 Desafio Final – Bootcamp Engenheiro(a) de Machine Learning
Este notebook apresenta a resolução completa do desafio final do módulo de **Engenharia de Machine Learning**, utilizando o dataset `cars.csv`.  
O pipeline segue os **7 passos fundamentais do Engenheiro de Machine Learning**, contemplando desde a análise exploratória até a modelagem supervisionada.


## 🥇 Passo 1 – Compreensão do problema e dos dados

In [ ]:

import pandas as pd
import numpy as np

# Leitura do dataset (ajuste o caminho conforme o seu ambiente)
df = pd.read_csv('/content/drive/MyDrive/cars.csv')

# Visualizações iniciais
print("Formato do dataset:", df.shape)
print("\nColunas disponíveis:\n", df.columns.tolist())
display(df.head())
print("\nInformações gerais:")
df.info()
display(df.describe())


## 🧩 Passo 2 – Coleta e exploração inicial do dataset

In [ ]:

# Verificação de valores ausentes e duplicados
print("Valores ausentes por coluna:")
print(df.isnull().sum())

print("\nLinhas duplicadas:", df.duplicated().sum())

# Valores únicos por coluna
print("\nValores únicos por coluna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")

# Identificação de tipos de dados
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print("\nVariáveis numéricas:", num_cols)
print("Variáveis categóricas:", cat_cols)


## 🧹 Passo 3 – Pré-processamento (limpeza e normalização com `pd.to_numeric`)

In [ ]:

from sklearn.preprocessing import StandardScaler

# Conversão segura de colunas numéricas com errors='coerce'
for col in ['cubicinches', 'weightlbs']:
    df[col] = df[col].astype(str).str.replace(',', '').str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Substituir valores nulos pela média
df['cubicinches'].fillna(df['cubicinches'].mean(), inplace=True)
df['weightlbs'].fillna(df['weightlbs'].mean(), inplace=True)

# Criar coluna de eficiência com base no mpg
df['efficiency'] = pd.cut(df['mpg'], bins=[0, 20, 30, df['mpg'].max()], labels=['Low', 'Medium', 'High'])

# Normalização das variáveis numéricas
scaler = StandardScaler()
num_features = ['cylinders', 'cubicinches', 'hp', 'weightlbs', 'time-to-60']

df_scaled = df.copy()
df_scaled[num_features] = scaler.fit_transform(df[num_features])

print("Tipos de dados após conversão:\n", df_scaled.dtypes)
display(df_scaled.head())


## 🔍 Passo 4 – Análise exploratória e correlações

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Matriz de correlação
corr = df_scaled[['mpg', 'cylinders', 'cubicinches', 'hp', 'weightlbs', 'time-to-60']].corr()

print("Correlação com a variável mpg:\n")
print(corr['mpg'].sort_values(ascending=False))

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='RdBu', center=0)
plt.title('Matriz de Correlação – Variáveis Numéricas')
plt.show()


## 🧭 Passo 5 – Redução de dimensionalidade com PCA

In [ ]:

from sklearn.decomposition import PCA

X = df_scaled[['cylinders', 'cubicinches', 'hp', 'weightlbs', 'time-to-60']]
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

explained_var = pca.explained_variance_ratio_
print("Variância explicada por componente:\n", explained_var)
print("\nVariância acumulada:", explained_var.cumsum())

plt.figure(figsize=(8,5))
plt.plot(range(1, len(explained_var)+1), explained_var.cumsum(), marker='o')
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Variância Acumulada Explicada')
plt.title('PCA – Variância Explicada Acumulada')
plt.grid(True)
plt.show()


## 🧮 Passo 6 – Clusterização com K-Means

In [ ]:

from sklearn.cluster import KMeans

X_pca2 = X_pca[:, :2]
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca2)

df_scaled['cluster'] = clusters

plt.figure(figsize=(8,6))
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=clusters, cmap='viridis', s=60)
plt.title('Clusters de Veículos (K-Means + PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.grid(True)
plt.show()

print("Distribuição de veículos por cluster:")
print(df_scaled['cluster'].value_counts())


## 🧠 Passo 7 – Modelagem supervisionada e avaliação

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = df_scaled[['cylinders', 'cubicinches', 'hp', 'weightlbs', 'time-to-60']]
y = df_scaled['efficiency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

log_reg = LogisticRegression(random_state=42, max_iter=500)
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)

print("=== Árvore de Decisão ===")
print("Acurácia:", accuracy_score(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

print("\n=== Regressão Logística ===")
print("Acurácia:", accuracy_score(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

import seaborn as sns
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.heatmap(confusion_matrix(y_test, y_pred_tree), annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Árvore de Decisão')
sns.heatmap(confusion_matrix(y_test, y_pred_log), annot=True, fmt='d', cmap='Greens', ax=axes[1])
axes[1].set_title('Regressão Logística')
plt.show()
